In [24]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('../Dados/CHURN_CREDIT_MOD14_TRATADO.csv', delimiter = ',')
df.head(10)

,Credit_Score,Pais,Genero,Idade,Tempo_Credito,Balanco,Qtd_Produtos,Possui_Cartao,Membro_Ativo,Salario_Anual,Churn
0,677,SPAIN,Male,18.0,5,134796.87,2,1.0,0.0,10787603.0,0.0
1,644,SPAIN,Female,18.0,7,0.00,1,0.0,1.0,10787603.0,0.0
2,641,SPAIN,Male,18.0,1,0.00,2,0.0,1.0,10787603.0,0.0
3,826,SPAIN,Male,18.0,2,0.00,2,0.0,1.0,10787603.0,0.0
4,774,SPAIN,Female,18.0,1,185891.54,1,1.0,0.0,10787603.0,0.0
5,688,SPAIN,Female,18.0,2,0.00,2,1.0,0.0,10787603.0,0.0
6,803,SPAIN,Male,18.0,6,0.00,1,0.0,0.0,10787603.0,1.0
7,578,SPAIN,Female,18.0,8,71843.15,2,0.0,1.0,10787603.0,0.0
8,598,SPAIN,Male,18.0,6,0.00,2,0.0,1.0,10787603.0,0.0
9,624,SPAIN,Female,18.0,6,0.00,2,0.0,1.0,10787603.0,0.0


### Verificando a coluna Balanço...

In [14]:
df_nao_zero = df[df['Balanco'] != 0]

fig = px.box(
    df_nao_zero, # ---> Fonte de dados
    y = 'Balanco', # ---> Coluna de plotagem
    title = 'Boxplot da coluna Balanço (Excluindo valores zerados)',
    labels = {'Balanco': 'Valores'}
)
fig.show()

Filtrando apenas os valores que estão acima do Upper Fence (Limite Superior) 186k+

*Validando se são dados legitimos ou possíveis erros de digitação

In [15]:
df[df['Balanco'] > 186000]

,Credit_Score,Pais,Genero,Idade,Tempo_Credito,Balanco,Qtd_Produtos,Possui_Cartao,Membro_Ativo,Salario_Anual,Churn
315,673,SPAIN,Female,23.0,1,186813.76,2,0.0,1.0,10787603.0,0.0
939,498,SPAIN,Male,26.0,1,198113.86,1,1.0,0.0,10787603.0,0.0
1142,771,SPAIN,Female,26.0,5,193354.86,2,1.0,0.0,10787603.0,0.0
1174,564,SPAIN,Male,26.0,3,187294.46,1,0.0,1.0,10787603.0,0.0
1866,850,SPAIN,Male,28.0,9,206014.94,2,0.0,1.0,10787603.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
97828,550,FRANCE,Female,58.0,4,207034.96,4,1.0,0.0,15769476.0,1.0
98106,662,FRANCE,Female,59.0,7,187911.55,1,0.0,0.0,235315.0,1.0
98695,539,FRANCE,Male,62.0,4,186825.57,1,1.0,1.0,12782525.0,1.0
98869,682,FRANCE,Male,63.0,4,207034.96,1,0.0,0.0,15739757.0,1.0


Vendo sua representação nos registros...

In [17]:
print(f'Representam cerca de {(len(df[df['Balanco'] > 186000]) / len(df)) * 100:.2f}% dos dados')

Representam cerca de 0.23% dos dados


Como a coluna possui mais de 50% dos seus registros zerados, seria interessante fazer a remoção dela para evitar enviesamento do modelo. Por enquanto, não vou mexer nela

**Exemplo, caso eu optasse por trocar os valores zerados pela média ou mediana**

In [19]:
# CRIANDO UMA CÓPIA DO DATAFRAME
df_exemplo = df.copy()

# CRIANDO OS QUARTIS DO BOXPLOT E CALCULANDO O IQR
Q1 = df_exemplo['Balanco'].quantile(0.25)
Q3 = df_exemplo['Balanco'].quantile(0.75)
IQR = Q3 - Q1 # ---> INTERVALO INTERQUARTIL: DIF ENTRE O PRIMEIRO QUARTIL E O TERCEIRO QUARTIL

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

df_sem_outliers = df_exemplo[(df_exemplo['Balanco'] >= limite_inferior) & (df['Balanco'] <= limite_superior)]

# CALCULANDO A MEDIANA COM REGISTROS SEM OUTLIERS
mediana_sem_outliers = df_sem_outliers['Balanco'].median()

# SUBSTITUINDO OS VALORES
df_exemplo.loc[ # .loc localiza valores específicos
    df_exemplo['Balanco'] > 186000,  # teste lógico do .loc
    'Balanco'] = mediana_sem_outliers # seleciona apenas a coluna 'Balanco' e atribui o valor da variável mediana_sem_outliers



### Coluna Qtd Produtos

In [21]:
fig = px.box(df, y = 'Qtd_Produtos', title = 'Boxplot Qtde_Produtos')
fig.show()

Vendo a distribuição das categorias da coluna

In [23]:
(df['Qtd_Produtos'].value_counts(normalize = True) * 100).round(2)

Qtd_Produtos
2    51.00
1    46.96
3     1.73
4     0.31
Name: proportion, dtype: float64

Como se trata de poucos valores e todos muito próximos, o 4 não pode ser tratado como um outlier nessa coluna.

Caso a coluna tivesse valores como 10 ou 20, seria ideal optar pela remoção desses valores por muito provavelmente se tratar de erros de digitação